## Generate unique for each transactions for identification


In [1]:
import pandas as pd
import numpy as np
import category_encoders
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

%load_ext autoreload
%autoreload 2

In [2]:
transactions_df = pd.read_parquet('../../artifacts/20201005/transactions_df_unlabeled_with_tmx.parquet')

### Add ID

In [5]:
transactions_df.sort_values(by=['business_account_number', 
                                'transaction_datetime'],
                            inplace=True)

In [22]:
transactions_df.head()['transaction_datetime'].apply(lambda x: str(int(x.timestamp())))

0    1513182391
1    1513727720
2    1513727720
3    1513727720
4    1513727720
Name: transaction_datetime, dtype: object

In [37]:
transactions_df['group_rank'] = transactions_df.groupby('business_account_number')['transaction_datetime'].rank('first').astype(int)

In [39]:
transactions_df['transaction_id'] = (transactions_df['business_account_number'].astype(str) 
                                     + '-' + transactions_df['transaction_datetime'].apply(lambda x: str(int(x.timestamp())))
                                     + '-' + transactions_df['group_rank'].astype(str))

In [40]:
transactions_df['transaction_id'].head()

0    410000007044-1513182391-1
1    410000007044-1513727720-2
2    410000007044-1513727720-3
3    410000007044-1513727720-4
4    410000007044-1513727720-5
Name: transaction_id, dtype: object

In [41]:
transactions_df['transaction_id'].nunique(), transactions_df.shape

(15295228, (15295228, 201))

### Add Features

In [42]:
# 'max_withdrawals_30d', 'dollar_val_returns_3d', 'max_deposits_30d', 'dollar_val_returns'

In [ ]:
transactions_df['max_withdrawals_30d'] = transactions_df.groupby('business_account_number') \
                                                        .rolling('30d', min_periods=1, 
                                                                 on='transaction_datetime') \
                                                        ['withdrawal_transaction_amount'].min().values
transactions_df['max_deposits_30d'] = transactions_df.groupby('business_account_number') \
                                                     .rolling('30d', min_periods=1, 
                                                              on='transaction_datetime') \
                                                     ['deposit_transaction_amount'].max().values


In [44]:
transactions_df['return_dollar_amount'] = transactions_df['is_return'] * transactions_df['tamt_adjusted']
transactions_df['dollar_val_returns'] = transactions_df.groupby('business_account_number')['return_dollar_amount'].cumsum()
transactions_df['dollar_val_returns_3d'] = transactions_df.groupby('business_account_number') \
                                                          .rolling('3d', min_periods=1, 
                                                                   on='transaction_datetime') \
                                                          ['return_dollar_amount'].sum().values

In [ ]:
transactions_df.to_parquet('../../artifacts/20201005/transactions_df_unlabeled_with_tmx_n_id.parquet')